In [5]:
x = Int8[5, 7, 3]
# type, length, address memory (called pointer)
eltype(x), length(x), pointer(x)

(Int8, 3, Ptr{Int8} @0x0000000119bccd38)

In [17]:
# DONT DO THIS
p = pointer(x)
Base.unsafe_load(p+2) # address + 2 gives 3rd entry

3

In [22]:
x = Int16[5, 7, 3]


3-element Vector{Int16}:
 5
 7
 3

In [28]:
p = pointer(x)
T = eltype(x) # type of each entry Int16
s = sizeof(T)
Base.unsafe_load(p + s)

7

In [30]:
x[2] = 11

11

In [31]:
x

3-element Vector{Int16}:
  5
 11
  3

## Matrix (dense)

In [34]:
A = [1 2;
     3 4;
     5 6]

# storage is same as a vector
vec(A)

6-element Vector{Int64}:
 1
 3
 5
 2
 4
 6

In [37]:
# a Matrix consists of 3 things:
eltype(A), size(A), pointer(A)

(Int64, (3, 2), Ptr{Int64} @0x0000000118333290)

## Matrix-vector mult.
# Alg 1 (by rows)

In [38]:
using LinearAlgebra

In [41]:
# inbuilt way

n = 5
A = randn(n,n) # n x n matrix with normal entries
x = randn(n)
A*x

5-element Vector{Float64}:
  3.6849885779795146
 -3.1740904211079393
 -5.02530716150381
  1.3563454318235286
  2.003051297982582

In [46]:
function mul_rows(A, x)
    # T is usually Float64
    T = promote_type(eltype(A), eltype(x))
    m,n = size(A) # dimensions of the matrix
    if n != length(x) #!= means not-equal
        error("incompatible")
    end
    b = zeros(T, m) # vector of all zeros
    for k = 1:m
        for j = 1:n
            b[k] = b[k] + A[k,j]*x[j]
        end
    end
    b
end

# alg 2
function mul_cols(A, x)
    # T is usually Float64
    T = promote_type(eltype(A), eltype(x))
    m,n = size(A) # dimensions of the matrix
    if n != length(x) #!= means not-equal
        error("incompatible")
    end
    b = zeros(T, m) # vector of all zeros
    for j = 1:n    
        for k = 1:m
            b[k] = b[k] + A[k,j]*x[j]
        end
    end
    b
end


A*x ≈ mul_rows(A,x) ≈ mul_cols(A,x)

true

In [53]:
n = 10000
A = randn(n,n)
x = randn(n)
println("inbulit")
@time A*x
println("by rows")
@time mul_rows(A,x)
println("by cols")
@time mul_cols(A,x)

inbulit
  0.034237 seconds (2 allocations: 78.172 KiB)
by rows
  0.755761 seconds (2 allocations: 78.172 KiB)
by cols
  0.077928 seconds (2 allocations: 78.172 KiB)


10000-element Vector{Float64}:
  -91.32837196728642
  -66.37156332549786
  -38.368867859098586
   95.98681803870481
 -162.49171777688417
 -128.77564970029033
   55.66592206334241
  -28.39291854484476
  -55.76962136905032
 -183.00758218995844
   68.22537677260885
   -1.8396974503482295
  -60.04561899915768
    ⋮
    1.5129231547920376
  -38.25097737332226
 -119.7293539012312
   78.21149588048314
  261.6888764599231
  -77.97595593391188
   59.61142149692241
 -221.5812251829955
  -33.589459647925516
 -171.14954542238306
  -76.58026627422821
   31.205822037898884

# Triangular matrices

In [55]:
A = [1 2 3; 4 5 6; 7 8 9]
U = UpperTriangular(A) # wraps the matrix A

3×3 UpperTriangular{Int64, Matrix{Int64}}:
 1  2  3
 ⋅  5  6
 ⋅  ⋅  9

In [57]:
U[1,3] = 6
U

3×3 UpperTriangular{Int64, Matrix{Int64}}:
 1  2  6
 ⋅  5  6
 ⋅  ⋅  9

In [58]:
A # modifying U also modifes A

3×3 Matrix{Int64}:
 1  2  6
 4  5  6
 7  8  9

In [60]:
fieldnames(typeof(U))
U.data

3×3 Matrix{Int64}:
 1  2  6
 4  5  6
 7  8  9

In [67]:
n = 10000
A = randn(n,n)
U = UpperTriangular(A)
b = randn(n)
@time U \ b # 10x faster by using back-substitution
@time A \ b; # uses matrix-factorization A = P * L *U

  0.017019 seconds (2 allocations: 78.172 KiB)
  6.802189 seconds (6 allocations: 763.092 MiB, 1.08% gc time)


In [68]:
function ldiv(U::UpperTriangular, b) # our implementation of U\b using backsub
    T = promote_type(eltype(U), eltype(b))
    m,n = size(U) # note m == n
    x = zeros(T, n)
    for k = n:-1:1 # compute x[n] first
        r = b[k] # r is a "dummy variabl"
        for j = k+1:n
            r = r - U[k,j]*x[j] # note we computed x[j] for j > k
        end
        x[k] = r/U[k,k]
    end
    x
end

ldiv (generic function with 1 method)

In [88]:
n = 1000
# A = randn(n,n)  This is VERY INACCURATE!! WHY???
A = randn(n,n) + 100I
U = UpperTriangular(A)
x = randn(n)
b = U*x

x ≈ U\b ≈ ldiv(U,b)

true

In [87]:
minimum(abs.(diag(A)))

0.0005342043291436943

# 3. Banded Matrices

In [89]:
d = [1,2,3,4]
D = Diagonal(d) # stored as a vector, l = u = 0

4×4 Diagonal{Int64, Vector{Int64}}:
 1  ⋅  ⋅  ⋅
 ⋅  2  ⋅  ⋅
 ⋅  ⋅  3  ⋅
 ⋅  ⋅  ⋅  4

In [90]:
d = [1,2,3,4,5]
u = [5,6,7,8]
U = Bidiagonal(d, u, :U) # stores two vectors and whether upper or loweer

5×5 Bidiagonal{Int64, Vector{Int64}}:
 1  5  ⋅  ⋅  ⋅
 ⋅  2  6  ⋅  ⋅
 ⋅  ⋅  3  7  ⋅
 ⋅  ⋅  ⋅  4  8
 ⋅  ⋅  ⋅  ⋅  5

In [91]:
d = [1,2,3,4,5]
u = [5,6,7,8]
l = [9,10,11,12]
T = Tridiagonal(l, d, u) # store 3 vectors

# Tridiagonal come up for solving ODEs

5×5 Tridiagonal{Int64, Vector{Int64}}:
 1   5   ⋅   ⋅  ⋅
 9   2   6   ⋅  ⋅
 ⋅  10   3   7  ⋅
 ⋅   ⋅  11   4  8
 ⋅   ⋅   ⋅  12  5

In [99]:
n = 10_000_000 # _ means ,
d = randn(n) .+ 1000 # add 1000 to diag so its "stable"
u = randn(n-1)
U = Bidiagonal(d, u, :U) # stores two vectors and whether upper or loweer
b = randn(n)
@time U \ b; # computational cost scales like n

  0.099128 seconds (2 allocations: 76.294 MiB)


In [103]:
fieldnames(typeof(U))
U.uplo # == 'U'

'U': ASCII/Unicode U+0055 (category Lu: Letter, uppercase)

In [105]:
function ldiv(U::Bidiagonal, b) # our implementation of U\b using backsub
    T = promote_type(eltype(U), eltype(b))
    m,n = size(U) # note m == n
    x = zeros(T, n)
    if U.uplo == 'U'
        for k = n:-1:1 # compute x[n] first
            r = b[k] # r is a "dummy variabl"
            for j = k+1:min(n,k+1) # only access non-zeros of U
                r = r - U[k,j]*x[j] # note we computed x[j] for j > k
            end
            x[k] = r/U[k,k]
        end
    else
        error("Not implemented")
    end
    x
end

ldiv (generic function with 2 methods)

In [107]:
n = 1000 # _ means ,
d = randn(n) .+ 1000 # add 1000 to diag so its "stable"
u = randn(n-1)
U = Bidiagonal(d, u, :U) # stores two vectors and whether upper or loweer
b = randn(n)
U \ b ≈ ldiv(U,b)

true